In [1]:
import faiss
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from pathlib import Path
from collections import defaultdict

/media/data/inno_courses/DLS/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model = SentenceTransformer('cointegrated/rubert-tiny2') # load embedding model

In [3]:
index = faiss.read_index("embeddings/embeddings.index")  # read index from file

In [ ]:
embeddings = np.load("embeddings/embeddings.npy", allow_pickle=True) # load ready embeddings from our dataset

In [34]:
embeddings.shape

(49727, 312)

In [ ]:
# dataset read
chunk_files = sorted(Path('.').glob('dataset/preproc_data*.parquet'))
dfs = [pd.read_parquet(f) for f in chunk_files]
df = pd.concat(dfs, ignore_index=True)

In [ ]:
# create class (only 2 numbers in classifierByIPS)
df[['classifier_code', 'classifier_name']] = df['classifierByIPS'].str.split('$', n=1, expand=True)
df['classifier_level2'] = df['classifier_code'].str.extract(r'^(\d{3}\.\d{3})')
print(df['classifier_level2'].value_counts())

classifier_level2
010.140    17889
010.070     5480
210.010     2896
210.020     2102
020.010     1997
           ...  
140.030        1
100.030        1
090.020        1
070.020        1
050.050        1
Name: count, Length: 158, dtype: int64


# Ranking metrics functions

In [38]:
def precision_at_k(relevant, k):
    return np.sum(relevant[:k]) / k

def recall_at_k(relevant, total_relevant, k):
    if total_relevant == 0:
        return 0.0
    return np.sum(relevant[:k]) / total_relevant


def hits_at_k(relevant, k):
    return 1.0 if np.sum(relevant[:k]) > 0 else 0.0

def mrr(relevant):
    for idx, rel in enumerate(relevant, 1):
        if rel:
            return 1.0 / idx
    return 0.0

def dcg(relevant, k):
    relevant = np.asarray(relevant)[:k] 
    if len(relevant) == 0:
        return 0.0
    discounts = np.log2(np.arange(2, len(relevant) + 1))
    return relevant[0] + np.sum(relevant[1:] / discounts[:len(relevant)-1])

def ndcg_at_k(relevant, k):
    relevant = np.asarray(relevant)[:k] 
    ideal_relevant = np.sort(relevant)[::-1] 
    idcg = dcg(ideal_relevant, k)
    if idcg == 0:
        return 0.0
    return dcg(relevant, k) / idcg

def average_precision_at_k(relevant, k):
    hits = 0
    sum_precisions = 0.0
    for i in range(k):
        if relevant[i]:
            hits += 1
            sum_precisions += hits / (i + 1)
    if hits == 0:
        return 0.0
    return sum_precisions / hits

In [39]:
def evaluate_index(index, embeddings, df, k=10, n_eval=100):
    precisions = []
    recalls = []
    hits = []
    mrrs = []
    ndcgs = []
    aps = []

    class_counts = df['classifier_level2'].value_counts().to_dict()

    for _ in range(n_eval):
        # choose random query from embedding
        i = np.random.randint(0, len(embeddings))
        query = embeddings[i].reshape(1, -1)
        query_class = df.iloc[i]['classifier_level2']

        if not isinstance(query_class, str) or query_class == "UNKNOWN":
            continue

        _, topk = index.search(query, k+1)
        topk = topk[0][1:]  # except the same one

        topk_classes = df.iloc[topk]['classifier_level2'].values
        relevant = (topk_classes == query_class).astype(int)

        precisions.append(precision_at_k(relevant, k))
        total_relevant = class_counts.get(query_class, 0) - 1
        total_relevant = max(total_relevant, 0)
        recalls.append(recall_at_k(relevant, total_relevant, k))
        hits.append(hits_at_k(relevant, k))
        mrrs.append(mrr(relevant))
        ndcgs.append(ndcg_at_k(relevant, k))
        aps.append(average_precision_at_k(relevant, k))

    print("Evaluation results:")
    print(f"Precision@{k}: {np.mean(precisions):.3f}")
    print(f"Recall@{k}: {np.mean(recalls):.3f}")
    print(f"Hits@{k}:     {np.mean(hits):.3f}")
    print(f"MRR:          {np.mean(mrrs):.3f}")
    print(f"NDCG@{k}:     {np.mean(ndcgs):.3f}")
    print(f"MAP@{k}:      {np.mean(aps):.3f}")

In [40]:
evaluate_index(index, embeddings, df, 10, 100)

Evaluation results:
Precision@10: 0.643
Recall@10: 0.002
Hits@10:     0.947
MRR:          0.769
NDCG@10:     0.807
MAP@10:      0.745


### Test on query and class (50 pairs), but some classes are not in dataset range

In [6]:
test_query = [
    # Уголовное право и процесс
    "Как подать жалобу на незаконные действия следователя?",
    "Что считается смягчающими обстоятельствами в уголовном деле?",
    "Как обжаловать приговор суда по уголовному делу?",
    "Какие права у подозреваемого при задержании?",
    # Жилищное право
    "Как выселить недобросовестного квартиросъемщика?",
    "Как оформить перепланировку квартиры законно?",
    "Какие льготы по квартплате для пенсионеров?",
    "Как разделить лицевой счет в коммунальной квартире?",
    # Трудовое право
    "Как оформить увольнение по сокращению штатов?",
    "Какая ответственность за задержку зарплаты?",
    "Как доказать факт трудовых отношений?",
    "Какие гарантии у беременных сотрудниц?",
    # Налоги и бизнес
    "Как получить налоговый вычет за лечение?",
    "Какие налоги платит ИП на УСН?",
    "Как оспорить решение налоговой проверки?",
    "Какая ответственность за незаконную предпринимательскую деятельность?",
    # Семейное право
    "Как лишить родительских прав недобросовестного родителя?",
    "Как взыскать алименты в твердой сумме?",
    "Как оформить брачный договор?",
    "Какие права у отца после развода?",
    # Административные вопросы
    "Как обжаловать штраф ГИБДД?",
    "Какие документы нужны для оформления загранпаспорта?",
    "Как оформить временную регистрацию?",
    "Как получить разрешение на строительство дома?",
    # Финансы и кредиты
    "Как реструктуризировать кредит в банке?",
    "Какие права у заемщика при навязывании страховки?",
    "Как вернуть страховку по кредиту?",
    "Что делать при незаконном списании средств со счета?",
    # Интеллектуальная собственность
    "Как зарегистрировать товарный знак?",
    "Какая ответственность за пиратство в интернете?",
    "Как оформить авторские права на книгу?",
    "Что делать при нарушении патентных прав?",
    # Здравоохранение
    "Как получить квоту на высокотехнологичную операцию?",
    "Какие права у пациента в частной клинике?",
    "Как оспорить врачебную ошибку?",
    "Как оформить инвалидность?",
    # Земельное право
    "Как оформить в собственность заброшенный земельный участок?",
    "Какие налоги на землю под ИЖС?",
    "Как разделить земельный участок между наследниками?",
    "Как оспорить кадастровую стоимость участка?",
    # общее
    "Как правильно оформить возбуждение уголовного дела по факту мошенничества?",
    "Какие стадии проходит законопроект перед принятием в Госдуме?",
    "Можно ли изменить или отменить действующий нормативный акт, если он устарел?",
    "Какие права есть у собственника жилого помещения?",
    "Как организовать управление многоквартирным домом?",
    "Какие требования к розничной торговле в 2024 году?"
    "Как оформить кредит для сельскохозяйственного предприятия?",
    "Какие документы нужны для призыва на военную службу?",
    "Как уволиться с военной службы по собственному желанию?",
    "Какие межведомственные комиссии могут проверить бизнес?"
]

test_class = [
    # Уголовное право и процесс
    '180.060', '180.070', '180.080', '180.090',
    # Жилищное право
    '050.030', '050.040', '050.050', '050.060',
    # Трудовое право
    '070.010', '070.020', '070.030', '070.040',
    # Налоги и бизнес
    '080.010', '080.020', '080.030', '080.040',
    # Семейное право
    '040.010', '040.020', '040.030', '040.040',
    # Административные вопросы
    '020.020', '020.030', '020.040', '020.050',
    # Финансы и кредиты
    '080.110', '080.120', '080.130', '080.140',
    # Интеллектуальная собственность
    '130.010', '130.020', '130.030', '130.040',
    # Здравоохранение
    '140.010', '140.020', '140.030', '140.040',
    # Земельное право
    '060.010', '060.020', '060.030', '060.040',
    # общее
    '180.060', '010.140', '010.140', '030.090', 
    '050.020', '090.100', '080.110', '150.090', 
    '150.100', '020.010'
]

In [28]:
def evaluate_index_with_queries(model, index, test_query, test_class, df, k=10):
    # create embeddings from queries
    query_vecs = model.encode(test_query)
    query_vecs = query_vecs / np.linalg.norm(query_vecs, axis=1, keepdims=True)
    faiss.normalize_L2(query_vecs)
    
    # count classifierByIPS values
    class_counts = df['classifier_level2'].value_counts().to_dict()
    
    metrics = {
        'precision': [],
        'recall': [],
        'hits': [],
        'mrr': [],
        'ndcg': [],
        'map': []
    }
    
    for i, (query_vec, true_class) in enumerate(zip(query_vecs, test_class)):
        _, topk_indices = index.search(query_vec.reshape(1, -1), k)
        topk_indices = topk_indices[0]
        
        # get classes of top lows
        topk_classes = df.iloc[topk_indices]['classifier_level2'].values
        
        relevant = (topk_classes == true_class).astype(int)
        
        # count metrics
        metrics['precision'].append(precision_at_k(relevant, k))
        
        total_relevant = class_counts.get(true_class, 0)
        metrics['recall'].append(recall_at_k(relevant, total_relevant, k))
        metrics['hits'].append(hits_at_k(relevant, k))
        metrics['mrr'].append(mrr(relevant))
        metrics['ndcg'].append(ndcg_at_k(relevant, k))
        metrics['map'].append(average_precision_at_k(relevant, k))
    
    print("Evaluation results:")
    print(f"Precision@{k}: {np.mean(metrics['precision']):.3f}")
    print(f"Recall@{k}: {np.mean(metrics['recall']):.3f}")
    print(f"Hits@{k}:     {np.mean(metrics['hits']):.3f}")
    print(f"MRR:          {np.mean(metrics['mrr']):.3f}")
    print(f"NDCG@{k}:     {np.mean(metrics['ndcg']):.3f}")
    print(f"MAP@{k}:      {np.mean(metrics['map']):.3f}")
    
    return metrics

In [29]:
results = evaluate_index_with_queries(
    model=model,
    index=index,
    test_query=test_query,
    test_class=test_class,
    df=df,
    k=10
)

Evaluation results:
Precision@10: 0.031
Recall@10: 0.004
Hits@10:     0.061
MRR:          0.044
NDCG@10:     0.077
MAP@10:      0.033


### Test on query and class (50 pairs)

In [41]:
test_query = [
    # Законодательные процессы (010.xxx)
    "Как подать законопроект в Государственную Думу?",  # 010.140
    "Каков порядок внесения поправок в Конституцию?",  # 010.070
    "Какие документы нужны для регистрации политической партии?",  # 010.100
    "Как оформить запрос о предоставлении законодательной инициативы?",  # 010.090
    "Какие существуют виды нормативных правовых актов?",  # 010.060
    
    # Государственное управление (020.xxx)
    "Как получить лицензию на образовательную деятельность?",  # 020.010
    "Каков порядок проведения конкурса на госслужбу?",  # 020.030
    "Как оформить межведомственный запрос?",  # 020.040
    "Какие полномочия у муниципальных органов власти?",  # 020.050
    "Как обжаловать решение органа исполнительной власти?",  # 020.020
    
    # Гражданское право (030.xxx)
    "Как оформить договор купли-продажи недвижимости?",  # 030.030
    "Какие права у потребителя при возврате товара?",  # 030.120
    "Как составить брачный договор?",  # 030.050
    "Каков порядок наследования по закону?",  # 030.040
    "Как оформить доверенность на представление интересов?",  # 030.020
    
    # Финансы и налоги (080.xxx)
    "Как получить налоговый вычет за лечение?",  # 080.050
    "Какие налоги платит ИП на упрощенке?",  # 080.080
    "Как оформить кредит для малого бизнеса?",  # 080.110
    "Какая ответственность за неуплату налогов?",  # 080.060
    "Как вернуть излишне уплаченный налог?",  # 080.100
    
    # Трудовое право (070.xxx)
    "Как правильно оформить увольнение по сокращению?",  # 070.030
    "Какие льготы положены работающим пенсионерам?",  # 070.070
    "Как составить трудовой договор с удаленным работником?",  # 070.060
    "Какая минимальная зарплата в 2024 году?",  # 070.010
    "Как оформить отпуск по уходу за ребенком?",  # 070.050
    
    # Образование (060.xxx)
    "Какие документы нужны для поступления в вуз?",  # 060.020
    "Как получить лицензию на образовательную деятельность?",  # 060.010
    "Какие существуют виды аттестации педагогов?",  # 060.020
    "Как оформить академический отпуск?",  # 060.010
    "Какие льготы есть для студентов?",  # 060.020
    
    # Здравоохранение (140.xxx)
    "Как получить квоту на высокотехнологичную операцию?",  # 140.010
    "Какие права у пациента в частной клинике?",  # 140.020
    "Как оформить инвалидность?",  # 140.010
    "Какие льготы на лекарства у пенсионеров?",  # 140.030
    "Как обжаловать врачебную ошибку?",  # 140.020
    
    # Военная служба (150.xxx)
    "Какие документы нужны для призыва в армию?",  # 150.090
    "Как получить отсрочку от военной службы?",  # 150.020
    "Какие льготы у военных пенсионеров?",  # 150.060
    "Как оформить военную ипотеку?",  # 150.010
    "Какие выплаты при увольнении с военной службы?",  # 150.100
    
    # Интеллектуальная собственность (130.xxx)
    "Как зарегистрировать товарный знак?",  # 130.010
    "Какая ответственность за пиратство в интернете?",  # 130.020
    "Как оформить патент на изобретение?",  # 130.030
    "Что делать при нарушении авторских прав?",  # 130.040
    "Как защитить коммерческую тайну?",  # 130.020
    
    # Семейное право (040.xxx)
    "Как лишить родительских прав?",  # 040.080
    "Какие алименты положены на 2 детей?",  # 040.060
    "Как усыновить ребенка из детдома?",  # 040.010
    "Как разделить имущество при разводе?",  # 040.040
    "Какие права у отца после развода?"  # 040.040
]

test_class = [
    '010.140', '010.070', '010.100', '010.090', '010.060',
    '020.010', '020.030', '020.040', '020.050', '020.020',
    '030.030', '030.120', '030.050', '030.040', '030.020',
    '080.050', '080.080', '080.110', '080.060', '080.100',
    '070.030', '070.070', '070.060', '070.010', '070.050',
    '060.020', '060.010', '060.020', '060.010', '060.020',
    '140.010', '140.020', '140.010', '140.030', '140.020',
    '150.090', '150.020', '150.060', '150.010', '150.100',
    '130.010', '130.020', '130.030', '130.040', '130.020',
    '040.080', '040.060', '040.010', '040.040', '040.040'
]

In [42]:
results = evaluate_index_with_queries(
    model=model,
    index=index,
    test_query=test_query,
    test_class=test_class,
    df=df,
    k=10
)

Evaluation results:
Precision@10: 0.028
Recall@10: 0.005
Hits@10:     0.080
MRR:          0.035
NDCG@10:     0.045
MAP@10:      0.032


### Avg time to search top 10

In [43]:
import time

def measure_faiss_speed(index, embeddings, n_queries=100):
    total_time = 0.0
    for _ in range(n_queries):
        i = np.random.randint(0, len(embeddings))
        query = embeddings[i].reshape(1, -1)
        start = time.time()
        _ = index.search(query, 10)
        total_time += time.time() - start
    avg_time_ms = (total_time / n_queries) * 1000
    return avg_time_ms
measure_faiss_speed(index, embeddings)

7.291843891143799

In [ ]:
# Real query test

In [21]:
df.columns

Index(['pravogovruNd', 'issuedByIPS', 'docdateIPS', 'docNumberIPS',
       'doc_typeIPS', 'headingIPS', 'doc_author_normal_formIPS', 'signedIPS',
       'statusIPS', 'actual_datetimeIPS', 'actual_datetime_humanIPS',
       'is_widely_used', 'textIPS', 'classifierByIPS', 'keywordsByIPS',
       'text_clean', 'tokens', 'lemmatized_text', 'classifier_code',
       'classifier_name', 'classifier_level2'],
      dtype='object')

In [ ]:
query = "украли кошелек"
query_vec = model.encode([query]) # encode
query_vec = query_vec / np.linalg.norm(query_vec) # normalize

faiss.normalize_L2(query_vec)

k = 50
# find the closest
similarities, indices = index.search(query_vec, k)

# filter resilt from top 50 to top 5
results = []
for idx, sim in zip(indices[0], similarities[0]):
    if idx == -1:  # if idx not exist, skip
        continue
    
    row = df.iloc[idx]
    classifier = str(row['classifier_level2']).strip() if pd.notna(row['classifier_level2']) else "UNKNOWN"
    
    # add if not UNKNOWN
    if classifier != "UNKNOWN":
        results.append({
            'index': idx,
            'similarity': sim,
            'classifier': row['classifier_code'],
            'classifier_level2': classifier,
            'text': row['textIPS'],
            'heading': row['headingIPS']
        })

# group by classifier_level2 (2 numbers from classifier_code)
grouped = defaultdict(list)
for res in results:
    grouped[res['classifier_level2']].append(res)

top_results = []
# take from each grouped 1 with max similarity
if len(grouped) >= 5:
    for cls in sorted(grouped.keys(), key=lambda x: max(y['similarity'] for y in grouped[x]), reverse=True)[:5]:
        best_in_cls = max(grouped[cls], key=lambda x: x['similarity'])
        top_results.append(best_in_cls)

# take from each grouped 2 with max similarity
else:
    candidates = []
    for cls, items in grouped.items():
        items_sorted = sorted(items, key=lambda x: -x['similarity'])
        candidates.extend(items_sorted[:2])
    # return only top 5
    top_results = sorted(candidates, key=lambda x: -x['similarity'])[:5]

print("Топ-5 результатов с учетом классов:")
for i, res in enumerate(top_results, 1):
    print(f"{i}. Схожесть: {res['similarity']:.3f} | Класс: {res['classifier_level2']}")
    print(f"Текст: {res['text']}")

Топ-5 результатов с учетом классов:
1. Схожесть: 0.474 | Класс: 010.070
Текст:  
 ПРАВИТЕЛЬСТВО РСФСР 
 РАСПОРЯЖЕНИЕ 
 от 28 декабря 1991 г. N 239-р
 г. Москва 
1. Министерству экономики и финансов РСФСР отпустить в 1992-1993 годах Внешторгбанку РСФСР для продажи на экспорт 1,3 тонны золота в счет сверхплановой добычи производственным объединением "Лензолото" в 1991-1992 годах.
2. Внешторгбанку РСФСР перечислить средства, вырученные от реализации 1,3 тонны золота, в распоряжение администрации Иркутской области для закупки продовольствия, товаров первой необходимости и технологий по переработке сельскохозяйственной продукции.
3. Администрации Иркутской области возместить Внешторгбанку РСФСР стоимость проданного золота в советских рублях по расчетным ценам, действующим при сдаче золота в Государственный фонд драгоценных металлов и драгоценных камней РСФСР. 
Первый заместитель Председателя
Правительства Российской Федерации Г. Бурбулис 
 
2. Схожесть: 0.472 | Класс: 030.050
Текст:  
 ПРАВ